## Set up Google Colab environment

In [1]:
### Set colaboratory True to run in Google Colaboratory. 
colab = True

if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  ## Specify a directory in Google Drive
  dir = '/content/drive/My Drive/Colab Notebooks/Save_Restore_Model'
  os.chdir(dir)
  #os.getcwd()
  #os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
if colab:
  ## Check the uptime. (Google Colab reboots every 12 hours)
  !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
  ## Check the GPU info
  !nvidia-smi
  ## Check the Python version
  !python --version

Uptime is 11.2148 hours (40373.35 sec)
Fri Jan 25 09:59:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                     

## Import modules

In [3]:
import os
import glob
import numpy as np
import pandas as pd
from IPython.display import display

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.12.0'

## Set up data

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## Define model

In [0]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer='adam', 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  return model

## Fit model

In [7]:
### If resume_if_possible = True, fitting can resume using a model file. 
resume_if_possible = True 

### Set resume_if_possible = False to force fitting from scratch.
#resume_if_possible = False 

epochs = 100
log_file_path = 'training_log.csv'
model_file_path = 'model_epoch{epoch:06d}.hdf5'

model_file_list = glob.glob('model_epoch*.hdf5')

if resume_if_possible:
  resume_flag = (len(model_file_list) >= 1) 
  
if not resume_if_possible:
  resume_flag = False
  for f in model_file_list:
    os.remove(f)

if resume_flag:
  latest_model_file = model_file_list[-1]
  latest_epoch = int(latest_model_file[len('model_epoch'):-len('.hdf5')])
  ## Load the saved model
  model = keras.models.load_model(latest_model_file)

  score = model.evaluate(test_images,test_labels, verbose=0)
  print('Use {} to resume fitting. \nTest loss: {}   Test accuracy: {}'.format(latest_model_file, score[0], score[1]))

  if len(model_file_list) >= 2:
    ## Delete all model files excpet the latest to save space
    for f in model_file_list[:-1]:
      os.remove(f)

if not resume_flag:
  latest_epoch = 0
  ## Create a basic model instance
  model = create_model()
  os.remove(log_file_path)

model.summary()

## Create checkpoint callback
check_point_ = tf.keras.callbacks.ModelCheckpoint(filepath = model_file_path, 
                                                 monitor = 'val_acc',
                                                 verbose=1,
                                                 save_best_only = True,
                                                 mode='auto',
                                                 save_weights_only=False,
                                                 period = 1)

## Create early stopping callback
early_stopping_ = tf.keras.callbacks.EarlyStopping(monitor='val_acc', 
                                                   min_delta=0, 
                                                   patience=3, 
                                                   verbose=1, 
                                                   mode='auto', 
                                                   baseline=None)

## Create CSV logger callback
csv_logger_ = tf.keras.callbacks.CSVLogger(filename = log_file_path, separator=',',
                                           append = resume_flag)

## Fit 
model.fit(train_images, train_labels, epochs = epochs, initial_epoch = latest_epoch,
          validation_data = (test_images,test_labels),
          callbacks = [check_point_, early_stopping_, csv_logger_])

## Remove duplicates as save_best_only option of checkpoint is set to True.
pd.options.display.max_rows = 8
log_df = pd.read_csv(log_file_path)
log_wo_dup_df = log_df.drop_duplicates(subset = ['epoch'], keep='last').reset_index(drop=True)
display(log_wo_dup_df)
log_wo_dup_df.to_csv((log_file_path[:-len('.csv')] + '_processed.csv'), index=False)

print('\nFiles in the working directoy:')
display(os.listdir())

Use model_epoch000018.hdf5 to resume fitting. 
Test loss: 0.41082902467250826   Test accuracy: 0.877
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Train on 1000 samples, validate on 1000 samples
Epoch 19/100
 768/1000 [======================>.......] - ETA: 0s - loss: 0.0096 - acc: 1.0000
Epoch 00019: val_acc improved from -inf to 0.88000, saving model to model_epoch000019.hdf5
1000/1000 [==============================] - 0s 436us/step - loss

,epoch,acc,loss,val_acc,val_loss
0,0,0.665,1.200675,0.787,0.721675
1,1,0.879,0.415959,0.842,0.525714
2,2,0.925,0.284789,0.853,0.469079
3,3,0.952,0.217569,0.878,0.418832
...,...,...,...,...,...
18,18,1.000,0.008906,0.880,0.423306
19,19,1.000,0.008253,0.875,0.424606
20,20,1.000,0.006871,0.877,0.430436
21,21,1.000,0.006214,0.877,0.426869



Files in the working directoy:


['keras_fit_with_callbacks_mnist.ipynb',
 'training_log.csv',
 'training_log_processed.csv',
 'model_epoch000018.hdf5',
 'model_epoch000019.hdf5']

References:


https://keras.io/callbacks/

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint